In [1]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import MTPtg

from astropy.utils import iers
iers.conf.auto_download = False

test_message = "Rotator Software Re-verification"

In [2]:
d = salobj.Domain()
script = salobj.Controller("Script", index=65769)
rot = salobj.Remote(d, "MTRotator")
mtptg = salobj.Remote(d, "MTPtg")

await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task)

rotation DDS read queue is filling: 76 of 100 elements
motors DDS read queue is filling: 76 of 100 elements
electrical DDS read queue is filling: 76 of 100 elements
ccwFollowingError DDS read queue is filling: 77 of 100 elements


[None, None, None]

In [25]:
await asyncio.sleep(1.)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

RuntimeError: Error on cmd=cmd_standby, initial_state=3: msg='Command failed', ackcmd=(ackcmd private_seqNum=1556534223, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: You must use the clearError command or the engineering user interface to clear a rotator fault.')

In [4]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

[<State.ENABLED: 2>]

In [5]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [6]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

Test Sequence #1 - PositionSet and Move Commands

In [7]:
await rot.cmd_move.set_start(position=15.0)

Test Sequence #2 - Stop Command

In [9]:
await rot.cmd_move.set_start(position=50.0)
await asyncio.sleep(5.)
await rot.cmd_stop.start()

In [14]:
await rot.cmd_move.set_start(position=50.0)
await asyncio.sleep(0.1)
await rot.cmd_stop.start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1090256844, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Low-level controller in substate 1.0 instead of <EnabledSubstate.STATIONARY: 0>')

In [30]:
await rot.cmd_stop.start()

Test startTrack without track

In [12]:
await rot.cmd_move.set_start(position=0.0)

In [16]:
await rot.cmd_trackStart.start()

In [19]:
await rot.cmd_trackStart.start()
await rot.cmd_track.set_start(angle=0.0,velocity=0.068,tai=salobj.current_tai()+0.05)
await asyncio.sleep(0.1)
await rot.cmd_stop.start()
await asyncio.sleep(100.)
await rot.cmd_trackStart.start()

In [39]:
await rot.cmd_stop.start()

Test Sequence #4 - Track and TrackStart Commands

In [22]:
#tel = rot.tel_Application.get()
#cpos = tel.Position
#print(cpos)

vel = 0.068
dt = (95 + -85) / vel
dpos = vel * 0.1
steps = int(dt / 0.1)

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

for i in range(steps):
    if i ==0:
        print("Max steps is %d." %steps)
    pos = -85 - i*dpos
    await rot.cmd_track.set_start(
        angle=pos,
        velocity=vel,
        tai=salobj.current_tai(),
        timeout=10.
    )
    print(i)
    print(pos)
    await asyncio.sleep(0.05)

#await rot.cmd_stop.start(timeout=30.)


Max steps is 1470.
0
-85.0
1
-85.0068
2
-85.0136
3
-85.0204
4
-85.0272
5
-85.034
6
-85.0408
7
-85.0476
8
-85.0544
9
-85.0612
10
-85.068
11
-85.0748
12
-85.0816
13
-85.0884
14
-85.0952
15
-85.102
16
-85.1088
17
-85.1156
18
-85.1224
19
-85.1292
20
-85.136
21
-85.1428
22
-85.1496
23
-85.1564
24
-85.1632
25
-85.17
26
-85.1768
27
-85.1836
28
-85.1904
29
-85.1972
30
-85.204
31
-85.2108
32
-85.2176
33
-85.2244
34
-85.2312
35
-85.238
36
-85.2448
37
-85.2516
38
-85.2584
39
-85.2652
40
-85.272
41
-85.2788
42
-85.2856
43
-85.2924
44
-85.2992
45
-85.306
46
-85.3128
47
-85.3196
48
-85.3264
49
-85.3332
50
-85.34
51
-85.3468
52
-85.3536
53
-85.3604
54
-85.3672
55
-85.374
56
-85.3808
57
-85.3876
58
-85.3944
59
-85.4012
60
-85.408
61
-85.4148
62
-85.4216
63
-85.4284
64
-85.4352
65
-85.442
66
-85.4488
67
-85.4556
68
-85.4624
69
-85.4692
70
-85.476
71
-85.4828
72
-85.4896
73
-85.4964
74
-85.5032
75
-85.51
76
-85.5168
77
-85.5236
78
-85.5304
79
-85.5372
80
-85.544
81
-85.5508
82
-85.5576
83
-85.5644
84
-8

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1992137420, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: current position -90.00490654830934 not in range [-90.0, 90.0]')

In [23]:
print(pos)

-90.0048


In [34]:
print("Azimuth = 0 and rot_tel= 60 deg")

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(10, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print("rot_pa", rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    #targetInstance=MTPtg.TargetInstances.CURRENT,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    #rotPA=rot_pa.deg,
    #rotFrame=ATPtg.RotFrame.TARGET,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)
    
print("Waiting 30s")
await asyncio.sleep(30.)

await mtptg.cmd_stopTracking.start(timeout=10.)

await asyncio.sleep(0.1)

await rot.cmd_stop.start(timeout=10.)

Azimuth = 0 and rot_tel= 60 deg
Time error=-0.00 sec
Alt: 45.0 deg
Az: 0.0 deg
RotTel: 10.0 deg
ParaAng: 179.67537378625644 deg
rot_pa 9d40m31.3456s
Waiting 30s


Test Sequence #5 - Track and TrackStart Commands

In [37]:
print("Azimuth = 180 and rot_tel= -60 deg")

alt = 45. * u.deg
az = 180. * u.deg
rot_tel = Angle(-60, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    #targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    #rotPA=rot_pa.deg,
    #rotFrame=ATPtg.RotFrame.TARGET,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)
#print("Waiting 30s")
#await asyncio.sleep(30.)

await mtptg.cmd_stopTracking.start(timeout=10.)

await asyncio.sleep(0.1)

await rot.cmd_stop.start(timeout=10.)

Azimuth = 180 and rot_tel= -60 deg
Time error=-0.00 sec
Alt: 45.0 deg
Az: 180.0 deg
RotTel: -60.0 deg
ParaAng: 0.39141488761718085 deg
-239d36m30.9064s


Test Sequence #6 - configureVelocity

In [41]:
await rot.cmd_configureVelocity.set_start(vlimit=3.0)

In [42]:
#To be rejected
await rot.cmd_configureVelocity.set_start(vlimit=3.0)

In [43]:
await rot.cmd_configureVelocity.set_start(vlimit=0.5)

In [ ]:
tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

newPos = cpos + 10.

await rot.cmd_positionSet.set_start(angle=newPos)
await rot.cmd_move.start(timeout=10.)

Test Sequence #7 - configureAcceleration

In [44]:
await rot.cmd_configureAcceleration.set_start(alimit=2.)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=263533579, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: alimit=2.0 must be > 0 and <= 1.0')

In [45]:
await rot.cmd_configureAcceleration.set_start(alimit=0.05)

In [ ]:
tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

newPos = cpos + 10.

await rot.cmd_positionSet.set_start(angle=newPos)
await rot.cmd_move.start(timeout=10.)

Section 3.3.2 State Machine

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await rot.cmd_start.start(timeout=10.) #probably won't run these

In [ ]:
await rot.cmd_enable.start(timeout=10.)

In [ ]:
await rot.cmd_disable.start(timeout=10.)

In [ ]:
await rot.cmd_standby.start(timeout=10.)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

Additional Error Handling Commands

In [26]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [27]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await rot.cmd_configureVelocity.set_start(vlimit=3.5)

In [ ]:
await rot.cmd_configureAcceleration.set_start(alimit=1.0)